# Getting started

In [ ]:
import pycellin as pc

## Pycellin data structure

*simplified scheme of a model*

### Metadata

### FeatureDeclaration

### Data

#### Cell lineages

*scheme*

#### Cycle lineages

*scheme*

## Loading data

*from external tools*  
*creating from scratch*

## Modifications of the lineages

*add node, edge, lineage...*  
*remove...*

## Managing features

*Just the basics here, ref the features notebooks*  
[Managing features](./Managing%20features.ipynb)  
[Advanced - Custom features](Advanced%20-%20Custom%20features.ipynb)

## Export